In [1]:
from typing import Union

from datasets import Dataset, load_dataset
from peft import LoraConfig  # type: ignore
from pydantic import BaseModel, Field

import xverify as xv

In [2]:
class Tools(BaseModel):
    """
    Run a tool.
    """

    tool_use: xv.XMLToolUse[xv.calculator, xv.search] = Field(
        ..., description="The tool call to use"
    )


class FinalAnswer(BaseModel):
    """
    Return a final answer.
    """

    answer: int = Field(..., description="Final answer to the question")


class Reason_and_Act(BaseModel):
    scratchpad: str = Field(
        ...,
        description="Information from the Observation useful to answer the question",
    )
    reasoning: str = Field(
        ...,
        description="It describes your thoughts about the question you have been asked",
    )
    # response: Tools | FinalAnswer # this still doesn't work
    response: Union[Tools, FinalAnswer]


def tool_response_func(model: Reason_and_Act) -> dict | None:
    return xv.run_tools(model.response)


guided_schema = xv.GuidedSchema(
    Reason_and_Act,
    schema="xml",
    tool_response_func=tool_response_func,
    max_steps=10,
)

In [3]:
def format_prompt(
    prompt: str, system_prompt: str | None = None
) -> list[dict[str, str]]:
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    return messages


def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()


SYSTEM_PROMPT = f"""\
You are a helpful assistant, responding in XML structured output.

- Think step by step using the scratchpad and reasoning outputs. You have {guided_schema.max_steps - 1} steps to think before responding.
- Use the tools provided. DO NOT rely on your own knowledge when a tool is available to help you.
- Respond with a final answer only once your are absolutely sure you have the answer.

Respond with a XML object, following the schema below:

{guided_schema.doc}
"""

SYSTEM_PROMPT = """\
TEST PROMPT
"""

dataset: Dataset = load_dataset("openai/gsm8k", "main", split="train")  # type: ignore
dataset = dataset.map(lambda x: {
    'prompt': [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': x['question']}
    ],
    'answer': extract_hash_answer(x['answer'])
})

In [ ]:
def exact_answer_reward_func(completions, answer, **_) -> list[float]:
    """Reward function that checks if the final answer matches the expected answer."""

    def _check_answer(trajectory: list[dict[str, str]], answer: int) -> float:
        """Extract the last answer from a trajectory."""
        last_message = trajectory[-1]
        assert last_message["role"] == "assistant", "should be assistant"
        parsed: Reason_and_Act | None = guided_schema.parse(last_message["content"])  # type: ignore
        if parsed is None or not isinstance(parsed.response, FinalAnswer):
            return 0.0
        return 1.0 if parsed.response.answer == answer else 0.0

    return [_check_answer(c, a) for c, a in zip(completions, answer)]


model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model, tokenizer = xv.get_model_and_tokenizer(model_name)


run_name = "gsm8k-calculator-peft_" + model_name.split("/")[-1].lower()
training_args = xv.get_default_grpo_config(run_name, num_gpus=1)

peft_config = LoraConfig(
    r=16,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj"],
)


trainer = xv.GRPOGuidedTrainer(
    guided_schema=guided_schema,
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config,
    reward_funcs=[exact_answer_reward_func],
)
trainer.train()

Using Liger kernel
Applied Liger kernels to Qwen2


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/home/tom/fun/xverify/.venv/lib/python3.12/site-packages/trl/trainer/grpo_trainer.py:471: UserWarning: The requested device cuda:0 is also being used for training. For higher throughput and to avoid out-of-memory errors, it is recommended to use a dedicated device for vLLM. If this is intentional, you may ignore this warning but should adjust `vllm_gpu_memory_utilization` accordingly.
  warnings.warn(


INFO 03-13 14:43:27 __init__.py:207] Automatically detected platform cuda.
INFO 03-13 14:43:33 config.py:549] This model supports multiple tasks: {'reward', 'classify', 'score', 'generate', 'embed'}. Defaulting to 'generate'.
INFO 03-13 14:43:33 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda:0, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_e

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-13 14:43:34 model_runner.py:1115] Loading model weights took 0.9266 GB
INFO 03-13 14:43:36 worker.py:267] Memory profiling takes 0.82 seconds
INFO 03-13 14:43:36 worker.py:267] the current vLLM instance can use total_gpu_memory (23.58GiB) x gpu_memory_utilization (0.30) = 7.07GiB
INFO 03-13 14:43:36 worker.py:267] model weights take 0.93GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.44GiB; the rest of the memory reserved for KV Cache is 4.65GiB.
INFO 03-13 14:43:36 executor_base.py:111] # cuda blocks: 25399, # CPU blocks: 21845
INFO 03-13 14:43:36 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 12.40x
INFO 03-13 14:43:39 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

...

╭──────────────────────────────────────────────────── Step 0 ─────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Prompt                                          ┃ Completion                                       ┃ Reward ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ <Reason_and_Act>                                │ assistant: <Reason_and_Act>                      │   0.00 │ │
│ │ <scratchpad>                                    │ <scratchpad>                                     │        │ │
│ │  1. Identify the total number of tables and the │  1. Identify the total number of tables and the  │        │ │
│ │ average number of seats per table.              │ average number of seats per table.               │        │ │
│ │  2. Calculate the total number of seats in the  │  2. Calculate the total number of seats in the   │        │ │
│ │ cafeteria.                                      │ cafeteria.                                       │        │ │
│ │  3. Determine the number of seats that are      │  3. Determine the number of seats that are       │        │ │
│ │ usually left unseated.                          │ usually left unseated.                           │        │ │
│ │  4. Subtraction the number of unseated seats    │  4. Subtraction the number of unseated seats     │        │ │
│ │ from the total number of seats to find the      │ from the total number of seats to find the       │        │ │
│ │ number of seats usually taken.                  │ number of seats usually taken.                   │        │ │
│ │ </scratchpad>                                   │ </scratchpad>                                    │        │ │
│ │ <reasoning>                                     │ <reasoning>                                      │        │ │
│ │  1. The total number of tables is 15 and each   │  1. The total number of tables is 15 and each    │        │ │
│ │ table can seat 10 people, so the total number   │ table can seat 10 people, so the total number of │        │ │
│ │ of seats is 15 x 10 = 150.                      │ seats is 15 x 10 = 150.                          │        │ │
│ │  2. Usually, 1/10 of the seats are left         │  2. Usually, 1/10 of the seats are left          │        │ │
│ │ unseated, so the number of seats usually left   │ unseated, so the number of seats usually left    │        │ │
│ │ unseated is 150 x 1/10 = 15.                    │ unseated is 150 x 1/10 = 15.                     │        │ │
│ │  3. The number of seats usually taken is the    │  3. The number of seats usually taken is the     │        │ │
│ │ total number of seats minus the number of seats │ total number of seats minus the number of seats  │        │ │
│ │ usually left unseated, so it is 150 - 15 = 135. │ usually left unseated, so it is 150 - 15 = 135.  │        │ │
│ │ </reasoning>                                    │ </reasoning>                                     │        │ │
│ │ <response>                                      │ <response>                                       │        │ │
│ │ <Tools>                                         │ <Tools>                                          │        │ │
│ │ <tool_use>                                      │ <tool_use>                                       │        │ │
│ │ <search>                                        │ <search>                                         │        │ │
│ │ <query>                                         │ <query>                                          │        │ │
│ │ What is the number of seats usually taken in    │ What is the number of seats usually taken in the │        │ │
│ │ the school's cafeteria?                         │ school's cafeteria?                              │        │ │
│ │ </query>                                        │ </

/home/tom/fun/xverify/.venv/lib/python3.12/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


╭──────────────────────────────────────────────────── Step 0 ─────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Prompt                                          ┃ Completion                                       ┃ Reward ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ <Reason_and_Act>                                │ assistant: <Reason_and_Act>                      │   0.00 │ │
│ │ <scratchpad>                                    │ <scratchpad>                                     │        │ │
│ │   Created note on the first page. I have        │   Created note on the first page. I have written │        │ │
│ │ written out the problem and the reasoning.      │ out the problem and the reasoning.               │        │ │
│ │ </scratchpad>                                   │ </scratchpad>                                    │        │ │
│ │ <reasoning>                                     │ <reasoning>                                      │        │ │
│ │                                                 │                                                  │        │ │
│ │   Let's break it down step by step:             │   Let's break it down step by step:              │        │ │
│ │                                                 │                                                  │        │ │
│ │   - The first question is simple: Fiona         │   - The first question is simple: Fiona          │        │ │
│ │ completed 36 math questions in an hour.         │ completed 36 math questions in an hour.          │        │ │
│ │   - The next sentence tells us that Shirley     │   - The next sentence tells us that Shirley      │        │ │
│ │ completed twice as many questions as Fiona      │ completed twice as many questions as Fiona       │        │ │
│ │ within the same hour. Therefore, Shirley        │ within the same hour. Therefore, Shirley         │        │ │
│ │ completed \(2 \times 36\) questions.            │ completed \(2 \times 36\) questions.             │        │ │
│ │   - The third sentence states that Kiana        │   - The third sentence states that Kiana         │        │ │
│ │ completed half of the sum of Fiona and          │ completed half of the sum of Fiona and Shirley's │        │ │
│ │ Shirley's math questions. Therefore, Kiana      │ math questions. Therefore, Kiana completed       │        │ │
│ │ completed \(\frac{1}{2}(36 + 2 \times 36)\)     │ \(\frac{1}{2}(36 + 2 \times 36)\) questions.     │        │ │
│ │ questions.                                      │   - Finally, since they each did the same number │        │ │
│ │   - Finally, since they each did the same       │ of questions the following hour, the total       │        │ │
│ │ number of questions the following hour, the     │ number of math questions completed by each girl  │        │ │
│ │ total number of math questions completed by     │ over two hours is the sum of Fiona and Shirley's │        │ │
│ │ each girl over two hours is the sum of Fiona    │ questions from the first hour plus the sum of    │        │ │
│ │ and Shirley's questions from the first hour     │ Kiana's questions from the second hour.          │        │ │
│ │ plus the sum of Kiana's questions from the      │                                                  │        │ │
│ │ second hour.                                    │   So, the total number of questions completed by │        │ │
│ │                                                 │ all three girls over two hours is:               │        │ │
│ │   So, the total number of questions completed   │   \(36 + 2 \times 36 + \frac{1}{2}(36 + 2 \times │        │ │
│ │ by all three girls over two hours is:           │ 36) = 36 + 72 + \frac{1}{2}(36 + 72)\)           │        │ │
│ │   \(36 + 2 \times 36 + \frac{1}{2}(36 + 2       │   

[2025-03-13 14:44:39] INFO lib.rs:450: response: https://html.duckduckgo.com/html 200
/home/tom/fun/xverify/.venv/lib/python3.12/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


╭──────────────────────────────────────────────────── Step 0 ─────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Prompt                                          ┃ Completion                                       ┃ Reward ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ <Reason_and_Act>                                │ assistant: <Reason_and_Act>                      │   0.00 │ │
│ │ <scratchpad>                                    │ <scratchpad>                                     │        │ │
│ │ You are right. Let's walk through the           │ You are right. Let's walk through the            │        │ │
│ │ calculations step by step using the information │ calculations step by step using the information  │        │ │
│ │ provided:                                       │ provided:                                        │        │ │
│ │                                                 │                                                  │        │ │
│ │ 1. Given that there are 20 skeletons in the     │ 1. Given that there are 20 skeletons in the      │        │ │
│ │ graveyard.                                      │ graveyard.                                       │        │ │
│ │ 2. Half of these skeletons are adult women, so  │ 2. Half of these skeletons are adult women, so   │        │ │
│ │ there are \( \frac{20}{2} = 10 \) adult women.  │ there are \( \frac{20}{2} = 10 \) adult women.   │        │ │
│ │ 3. The remaining 10 skeletons are split evenly  │ 3. The remaining 10 skeletons are split evenly   │        │ │
│ │ between adult men and children, which means     │ between adult men and children, which means      │        │ │
│ │ there are \( \frac{10}{2} = 5 \) adults and \(  │ there are \( \frac{10}{2} = 5 \) adults and \(   │        │ │
│ │ \frac{10}{2} = 5 \) children.                   │ \frac{10}{2} = 5 \) children.                    │        │ │
│ │ 4. An adult woman has 20 bones in their body.   │ 4. An adult woman has 20 bones in their body.    │        │ │
│ │                                                 │                                                  │        │ │
│ │ Now, let's look at the bones of an adult man:   │ Now, let's look at the bones of an adult man:    │        │ │
│ │ - An adult man has 5 more bones than an adult   │ - An adult man has 5 more bones than an adult    │        │ │
│ │ woman.                                          │ woman.                                           │        │ │
│ │ - Therefore, an adult man has \( 20 + 5 = 25 \) │ - Therefore, an adult man has \( 20 + 5 = 25 \)  │        │ │
│ │ bones.                                          │ bones.                                           │        │ │
│ │                                                 │                                                  │        │ │
│ │ Next, let's find out how many bones are in a    │ Next, let's find out how many bones are in a     │        │ │
│ │ child:                                          │ child:                                           │        │ │
│ │ - A child has half as many bones as an adult    │ - A child has half as many bones as an adult     │        │ │
│ │ woman.                                          │ woman.                                           │        │ │
│ │ - Therefore, a child has \( \frac{20}{2} = 10   │ - Therefore, a child has \( \frac{20}{2} = 10 \) │        │ │
│ │ \) bones.                                       │ bones.                                           │        │ │
│ │                                                 │                                                  │        │ │
│ │ Finally, the total number of bones in the       │ Finally, the total number of bones in the        │        │ │
│ │ graveyard is:                                   │ gr